In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input  #gerekli olan kütüphaneleri (paketleri) içe aktarıyoruz
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from detector.non_max_suppression import non_maxi_suppression
import numpy as np
import cv2


def selective_search(image):  #selective search algoritmasının fonksiyonunu oluşturuyoruz
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)  #algoritmanın işlem yapacağı resmi aktarıyoruz
    ss.switchToSelectiveSearchQuality()  #algoritmamızı içe aktardık 
    rects = ss.process()

    return rects[:500] #ilk 500 değeri dönderiyoruz (x,y,w,h)
 
#model
model = ResNet50(weights='imagenet') 
image = cv2.imread('D:/examples_forDeepLearning/unknown/src/animals.jpg') #hedef resmimizi diskten alıyoruz
image = cv2.resize(image,(400,400)) #yeniden boyutandırıyoruz
(H,W) = image.shape[:2] #height ve width değerlerini alıyoruz

#selectice search 
rects = selective_search(image)

proposals = []
boxes = []
for (x,y,w,h) in rects:  #döndürdüğümüz ilk 500 değerde geziniyoruz

    if w / float(W) < 0.1 or h /float(H) < 0.1: #şartı sağlayan w ve h değerlerini işleme almıyoruz
        continue
    roi = image[y:y+h,x:x+w] #roi oluşturuyoruz
    roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)  # bgr formatından rgb formatına dönüştürüyoruz
    roi = cv2.resize(roi,(224,224)) #yeniden boyutlandırıyoruz

    roi = img_to_array(roi)  #görüntümüzü resnet için uygun hale getiriyoruz
    roi = preprocess_input(roi) 

    proposals.append(roi)
    boxes.append((x,y,w,h))

proposals = np.array(proposals)   #numpy arrayı formatına çeviriyoruz

#prediction
preds = model.predict(proposals)  #tahminleri dönderiyoruz (resnet50)
preds = imagenet_utils.decode_predictions(preds,top=1) 

labels = {}  # olasılık değerleri 0.8 ' dan yüksek olan sınıfları bu sözlük içine ekliyoruz
for (i,p) in enumerate(preds):
    
    (_,label,prob) = p[0] 
    if prob >= 0.8:
        (x,y,w,h) = boxes[i]
        box = (x,y,x+w,y+h)
        L = labels.get(label, [])
        L.append((box,prob))
        labels[label] = L

clone = image.copy() #orijinal resmimizin bozulmaması için  clone adlı bir kopya oluşturuyoruz
#print(labels.keys()) => # dict_keys(['eskimo_dog'])
for label in labels.keys():  #tekrarlayan sınırlayıcı kutuları önlemek için 
    
    for (box, prob) in labels[label]:
        boxes = np.array([p[0] for p in labels[label]]) 
        proba = np.array([p[1] for p in labels[label]])
        boxes = non_maxi_suppression(boxes,proba) #tekrarlayan sınırlayıcı kutuları önlüyoruz

        for (startX, startY,endX,endY) in boxes:  #önlenmiş sınırlayıcı kutular ve etiketleri clone görüntüsü üzerine çiziyoruz ve sınıflandırılmış nesnelerin etiketlerini ilgili sınırlayıcı kutuların yanına yazıyoruz
            cv2.rectangle(clone,(startX,startY),(endX,endY),(0,0,255),2)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(clone,label,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),2)
        cv2.imshow('after',clone)  #clone resmimizi imshowluyoruz
        if cv2.waitKey(1) == ord('q'): # eğer ki 'q' ya basılırsa döngüyü kırıyoruz
            break

cv2.destroyAllWindows() #tüm pencereleri kapat


